In [1]:
# ability to give a gift to a friend on instagram
# ability to create tweets-like feature on instagram
# create a marketplace for educators to sell their courses on instagram

In [2]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback

from IPython.display import display, Markdown
import wandb
from wandb.integration.langchain import WandbTracer
import os

In [3]:
string_template = """\
You are a tech product manager. You have to help the user create a Product Requirement Document based on the questions the user asks you. The user will ask you specific questions about each topic they want to be included in the PRD. 

Do not repeat the same information again and again. Answers to each question should be unique and not repetitive. By this I mean do not repeat any ideas or sentences. Do not copy statements and ideas from previous sections. Any ideas or examples should only be in accordance to the particular section.

Format your responses in Markdown mode with each topic being the ##Heading, and your answer being the content. Highlight important points in **bold**. Give the PRD a suitable #Title.

For reference, let us say there are 3 people - A, B, and C belonging to different age groups, professions, and geographies. A is a 20-year-old college student from India. B is a 40-year-old working professional from the US. C is a 60-year-old retired person from the UK.
If required, for that particular section, you can use any of these people as examples to explain your point.

You do not need to include these 3 people in every section. You can use them as examples only if required. You can also use other examples if you want to. You can also use yourself as an example if you want to.

Current conversation:
{history}
Human: {input}
AI: """

prompt = PromptTemplate(
    template=string_template,
    input_variables=["history", "input"],
)

prompts_list = [
    """Product Overview:
Define the Purpose and Scope of this product. It should include how different groups of users across ages, genders, and geographies can use this product. Include an overview of the product. Why should one use this product? Define the target audience and stakeholders in detail. Also, include the rationale behind having the particular group as the target audience. Explain the gap it is trying to fill as well - how it is different from and better than other similar products?""",
    """Product Objectives:
First, analyze whether the product objectives align with the company objectives. Think aloud. Explain your reasoning. Also, talk about why and how the business models of the product and company match. What company goals can the product help achieve - be it attracting customers, generating profits, or promoting the goodwill of the company? Also, explain how it would do this.""",
    """Market Research:
First, list out current and potential competitors. Current competitors should include already established businesses/products. Potential competitors should include products and businesses that aren’t yet popular or are still under development/ beta version. Also include major or minor differences between our product and the competitor products you have identified. Analyze how aspects of our product or competitor products are better for that particular aspect. How do the target customers different? Does our product better cater to current trends and expectations of the users? How? What should the product include to meet those trends and expectations.""",
    """Competitive Analysis Table:
Use all the above competitors to create a competitive analysis of these applications in a tabular form using the following points - user base, user region, different features supported, and pricing tiers. Don't limit yourself to these categories and think of other categories yourself. Return the output in a well-structured Markdown table""",
    """Feature Requirements:
What are some of the important features that should be implemented? Follow the MoSCoW format (Must have, Should have, Could have, Won’t have, along with why). How are we going to collect user inputs and use user data that we collect to make the product better and add other features?""",
    """Launch Strategy:
Compare US vs International markets for this product. Also, analyze this product and figure out what customer demographic is this product for. Based on these things, come up with a detailed launch strategy for the product. List the TAM vs SAM vs SOM. TAM or Total Available Market is the total market demand for a product or service. SAM or Serviceable Available Market is the segment of the TAM targeted by your products and services which is within your geographical reach. SOM or Serviceable Obtainable Market is the portion of SAM that you can capture.""",
    """User Stories:
Create user stories for the product. User stories are short, simple descriptions of a feature told from the perspective of the person who desires the new capability, usually a user or customer of the system. They typically follow a simple template: As a < type of user >, I want < some goal > so that < some reason >. For example, As a college student, I want to be able to share my notes with my friends so that I can help them with their studies.""",
    """Acceptance Criteria:
Define the quality of completeness required to be able to get to the MVP stage of this product.""",
    """Success Metrics:
How do we define success in this product? What are the KPIs to look out for? How are they measured? Why do those KPIs matter? How are we going to use these KPIs to make the product better?""",
    """Technical Feasibilities:
Outline the technical roadmap for this product. What mobile devices should this application be available for? What is a scalable and reliable tech stack which can be used for the frontend and the backend for this application?""",
    """Timeline:
Define the timeline for the product development. In addition to the timeline, what are the resources required to complete this project. Think about the resources required for each stage of the project, the number of employees required for each stage, and the time required for each stage."""
]


In [4]:
chat = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    openai_api_key=os.environ["OPENAI_API_KEY"],
)

wandb.init(
    project="chat-prd-gpt-4",
    config={
        "model": "gpt-4",
        "temperature": 0
    },
    entity="arihantsheth",
    name="Instagram Threads_prd_gpt-4",
)

memory = ConversationBufferMemory()

chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=prompt,
    verbose=False
)

total_tokens = 0
total_cost = 0.0

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arihantsheth. Use `wandb login --relogin` to force relogin


In [8]:
with get_openai_callback() as callback:

    initial_output = chain.predict(
        input="""\
I want to add the following new feature:
Instagram Threads.

Feature description: A new app by Instagram that allows users to post tweet-like content.

DO NOT START WRITING. WAIT FOR THE HUMAN TO WRITE "Start generating the PRD" BEFORE YOU START WRITING.
"""
    )

    total_tokens += callback.total_tokens
    total_cost += callback.total_cost

display(Markdown(f"{initial_output}"))

Sure, I'm ready to help you create a PRD for this new feature. Please let me know when you want to start.

In [9]:
output = ""
for i, prompt in enumerate(prompts_list):
    
    with get_openai_callback() as callback:

        output += chain.predict(
            input=prompt,
            callbacks=[WandbTracer()]
        )

        output += "\n\n"

wandb.log({"prd": output})
wandb.finish()

prd,# Product Requiremen...


In [7]:
output += chain.predict(
    input="Timeline: Define the timeline for the product development. In addition to the timeline, what are the resources required to complete this project. Think about the resources required for each stage of the project, the number of employees required for each stage, and the time required for each stage.",
    callbacks=[WandbTracer()]
)

output += "\n\n"

wandb: Streaming LangChain activity to W&B at https://wandb.ai/arihantsheth/productx/runs/mdniyd97
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


In [11]:
total_cost

0.01221

In [8]:
# with open("../generated_prds/Instagram Threads PRD Chat gpt-4.md", "w") as f:
#     f.write(output)

In [12]:
display(Markdown(output))

# Product Requirement Document: Instagram Threads

## Product Overview

The purpose of Instagram Threads is to provide a platform for users to share short, tweet-like content, expanding the scope of Instagram's social media offerings. This product is designed to cater to a wide range of users across different age groups, genders, and geographies. 

For instance, a 20-year-old college student from India (A) might use Instagram Threads to share quick thoughts or updates about their day, while a 40-year-old working professional from the US (B) might use it to share industry insights or professional updates. A 60-year-old retired person from the UK (C) might use it to share their thoughts on current events or personal anecdotes.

The rationale behind targeting such a diverse audience is the universal appeal of sharing thoughts and ideas. Instagram Threads aims to fill the gap between Instagram's photo-centric platform and the need for a space to share text-based content. 

The stakeholders for this product include the users, Instagram as a company, advertisers who might want to leverage this new feature, and third-party developers who might build additional features or apps around Instagram Threads.

The product stands out from similar offerings due to Instagram's already established user base and the seamless integration with the existing Instagram app. It offers a unique value proposition by combining the visual appeal of Instagram with the brevity and immediacy of a Twitter-like platform. 

**Why should one use this product?** Instagram Threads allows users to express themselves in a new, concise format, fostering a sense of community and encouraging more frequent engagement. It is a step towards making Instagram a more comprehensive social media platform.

## Product Objectives

The product objectives of Instagram Threads align well with the overarching objectives of Instagram as a company. Instagram's mission is to bring people closer to the things they love, and Instagram Threads does exactly that by providing a new way for users to express themselves and connect with others.

The business model of Instagram Threads also aligns with Instagram's existing model. Instagram primarily generates revenue through advertising, and Instagram Threads provides a new platform for advertisers to reach their target audience. The short, tweet-like format of Threads could potentially attract a different demographic of advertisers who prefer this concise format, thereby diversifying Instagram's advertising clientele.

Instagram Threads can help achieve several company goals. Firstly, it can attract new users who prefer text-based social media platforms, thereby expanding Instagram's user base. Secondly, it can increase user engagement as the new feature encourages more frequent posting. More engagement translates into more opportunities for advertising, which can boost profits. 

Lastly, Instagram Threads can promote the goodwill of the company by demonstrating Instagram's commitment to innovation and user satisfaction. By continuously evolving and introducing new features, Instagram shows that it values its users and is dedicated to improving their experience. 

In terms of how it would achieve these goals, Instagram Threads would leverage Instagram's existing user base and infrastructure. The feature would be integrated into the existing app, making it easy for current users to start using Threads. The new feature could also be promoted through Instagram's other platforms, such as Instagram Stories or IGTV, to increase awareness and adoption.

## Market Research

**Current Competitors:**

1. **Twitter:** Twitter is a major competitor as it already offers a platform for users to share short, tweet-like content. However, Instagram Threads differentiates itself by integrating with Instagram's visual platform, offering a unique blend of text and image-based content.

2. **Facebook Status Updates:** Facebook's status update feature is another competitor. However, Instagram Threads could appeal to users who prefer Instagram's user interface and community.

**Potential Competitors:**

1. **Emerging Social Media Platforms:** New social media platforms that are still in their development or beta stages could potentially become competitors. These platforms could offer similar features or entirely new ways of sharing content.

2. **Existing Platforms Introducing Similar Features:** Existing social media platforms could introduce similar features, becoming potential competitors. For example, Snapchat or LinkedIn could introduce a feature for sharing short, text-based content.

The target customers for Instagram Threads are existing Instagram users who want a platform for sharing short, text-based content, as well as new users who prefer this type of content. This differs from Twitter, which has a broader user base, and Facebook, which tends to attract an older demographic.

Instagram Threads aligns with current trends and user expectations by offering a platform for quick, concise communication. In today's fast-paced world, users appreciate being able to share and consume content quickly. Instagram Threads caters to this trend by offering a platform for sharing short, tweet-like content.

To meet these trends and expectations, Instagram Threads should include features that facilitate quick and easy content creation and sharing. This could include a simple user interface, integration with Instagram's existing features (like the ability to share photos or Stories), and tools for discovering and interacting with content from other users.

## Competitive Analysis Table

| Competitor | User Base | User Region | Different Features Supported | Pricing Tiers | Unique Selling Proposition |
|------------|-----------|-------------|------------------------------|---------------|----------------------------|
| Twitter | 330 million monthly active users | Worldwide | Tweeting, Retweeting, Commenting, Direct Messaging, Trending Topics, Lists | Free, with promoted tweets and ads for revenue | Real-time updates and discussions, wide user base |
| Facebook Status Updates | 2.8 billion monthly active users | Worldwide | Status Updates, Commenting, Liking, Sharing, Groups, Pages, Marketplace | Free, with targeted ads for revenue | Wide user base, comprehensive features for social networking |
| Emerging Social Media Platforms | Varies | Varies | Varies | Varies | Potential for innovative features and new user experiences |
| Existing Platforms Introducing Similar Features (e.g., Snapchat, LinkedIn) | Varies | Varies | Varies | Varies | Existing user base, potential for integration with current features |

## Feature Requirements

**Must Have:**

1. **Short Text Posts:** The core feature of Instagram Threads. Users must be able to create and share short, tweet-like posts. This is essential for the product's main function.

2. **Integration with Instagram:** Instagram Threads should be seamlessly integrated with the main Instagram app to leverage the existing user base and provide a cohesive user experience.

3. **User Interaction Features:** Users must be able to like, comment on, and share posts. These features are standard in social media platforms and are crucial for user engagement.

**Should Have:**

1. **Discoverability Features:** Features like trending topics or suggested users can help users discover new content and encourage engagement.

2. **Privacy Settings:** Users should have control over who can see their posts. This is important for user comfort and safety.

**Could Have:**

1. **Customizable User Interface:** Users could have the option to customize the look and feel of their Instagram Threads interface. This could increase user satisfaction and engagement.

2. **Integration with Other Social Media Platforms:** Users could have the option to share their Instagram Threads posts on other social media platforms. This could increase the reach of Instagram Threads and attract new users.

**Won't Have:**

1. **Long Form Content:** Instagram Threads won't support long-form content. The product is designed for short, tweet-like posts, and supporting long-form content could dilute this focus.

To collect user inputs and use user data, Instagram Threads could implement features like user surveys and feedback forms. User interaction data (like which posts get the most likes or comments) could also be analyzed to understand user preferences and trends. This data can be used to continuously improve the product and add new features that align with user needs and preferences.

## Launch Strategy

When comparing the US vs International markets for Instagram Threads, both present significant opportunities. The US market, with its high social media usage rates and familiarity with Instagram, could be an ideal place to initially launch and test the product. However, the international market, particularly in regions like India and Southeast Asia where social media usage is rapidly growing, also presents a significant opportunity for expansion.

The customer demographic for Instagram Threads is broad, encompassing anyone who uses social media and prefers a platform for sharing short, text-based content. This could include younger users who are already familiar with Instagram, as well as older users who are looking for a new way to express themselves online.

**Launch Strategy:**

1. **Phase 1 - Beta Testing:** Launch a beta version of Instagram Threads to a select group of users in the US. Collect feedback and make necessary adjustments.

2. **Phase 2 - US Launch:** After refining the product based on beta testing feedback, launch Instagram Threads to the entire US market. Promote the product through Instagram's other platforms and through targeted advertising.

3. **Phase 3 - International Expansion:** Once the product has been proven successful in the US market, begin expanding to key international markets. This could start with English-speaking countries, followed by localized versions for non-English speaking markets.

**TAM vs SAM vs SOM:**

- **TAM (Total Available Market):** This would be all social media users worldwide, estimated to be around 3.6 billion people.

- **SAM (Serviceable Available Market):** This would be all Instagram users, as Instagram Threads is integrated with the Instagram app. As of 2020, Instagram has over 1 billion monthly active users.

- **SOM (Serviceable Obtainable Market):** This would be a subset of Instagram users who are interested in sharing short, text-based content. An exact number would require further market research, but for the sake of estimation, let's say this is 10% of Instagram users, or 100 million people.

## User Stories

1. **As a college student (A),** I want to share quick updates about my day on Instagram Threads so that I can keep my friends informed about what I'm up to.

2. **As a working professional (B),** I want to share industry insights on Instagram Threads so that I can establish myself as a thought leader in my field.

3. **As a retired person (C),** I want to share my thoughts on current events on Instagram Threads so that I can engage in discussions with others.

4. **As an Instagram user,** I want to be able to easily switch between Instagram and Instagram Threads so that I can enjoy both visual and text-based content in one place.

5. **As a social media advertiser,** I want to be able to target my ads to Instagram Threads users so that I can reach a demographic that prefers short, text-based content.

6. **As a user concerned about privacy,** I want to control who can see my Instagram Threads posts so that I can share my thoughts comfortably.

7. **As an Instagram influencer,** I want to share my thoughts and ideas on Instagram Threads so that I can engage with my followers in a new way.

## Acceptance Criteria

To reach the Minimum Viable Product (MVP) stage for Instagram Threads, the product should meet the following criteria:

1. **Functionality:** The core feature of creating and sharing short, tweet-like posts should be fully functional. Users should be able to easily create a post, and the post should appear on their followers' feeds.

2. **Integration:** Instagram Threads should be seamlessly integrated with the main Instagram app. Users should be able to switch between Instagram and Instagram Threads without leaving the app.

3. **User Interaction:** Users should be able to like, comment on, and share posts on Instagram Threads. These features should work consistently and reliably.

4. **Performance:** Instagram Threads should load quickly and operate smoothly, without crashes or significant lag.

5. **Usability:** The user interface should be intuitive and easy to navigate, even for new users. The process of creating a post should be straightforward and require minimal steps.

6. **Security and Privacy:** Users should have control over who can see their posts. Privacy settings should be easy to access and adjust.

7. **Feedback Mechanism:** There should be a mechanism for users to provide feedback on the product. This could be a simple form within the app or a dedicated email address for feedback.

If Instagram Threads meets these criteria, it can be considered ready for the MVP stage. Further features and improvements can be added based on user feedback and data collected during the MVP stage.

## Success Metrics

Success for Instagram Threads can be defined by several Key Performance Indicators (KPIs):

1. **User Adoption:** The number of users who start using Instagram Threads. This can be measured by tracking the number of unique users who create a post on Instagram Threads. This KPI matters because it indicates the level of interest in and acceptance of the new feature.

2. **User Engagement:** The level of interaction with Instagram Threads posts. This can be measured by tracking metrics like the number of likes, comments, and shares per post. This KPI matters because it indicates how much users are engaging with the content on Instagram Threads.

3. **Retention Rate:** The percentage of users who continue to use Instagram Threads over time. This can be measured by tracking the number of active users over time and calculating the percentage who remain active. This KPI matters because it indicates whether users find the feature valuable enough to continue using it.

4. **Revenue Generation:** The revenue generated through advertising on Instagram Threads. This can be measured by tracking the revenue from ads displayed on Instagram Threads. This KPI matters because it indicates the financial viability of the new feature.

These KPIs can be used to make the product better by providing insights into how users are interacting with Instagram Threads and where improvements can be made. For example, if user engagement is low, it might indicate that users are finding it difficult to interact with posts, suggesting that the user interface could be improved. Similarly, if the retention rate is low, it might indicate that users are not finding the feature valuable, suggesting that additional features or adjustments might be needed.

## Technical Feasibilities

The technical roadmap for Instagram Threads could look something like this:

1. **Phase 1 - Development:** Develop the core features of Instagram Threads, including the ability to create and share short, text-based posts and the integration with the main Instagram app.

2. **Phase 2 - Testing:** Conduct thorough testing to ensure the functionality, performance, and security of Instagram Threads.

3. **Phase 3 - Beta Launch:** Launch a beta version of Instagram Threads to a select group of users and collect feedback.

4. **Phase 4 - Refinement:** Refine the product based on user feedback and prepare for the full launch.

5. **Phase 5 - Full Launch:** Launch Instagram Threads to all users and continue to monitor performance and collect feedback for further improvements.

Instagram Threads should be available for both iOS and Android devices to reach the widest possible audience. 

For the tech stack, Instagram currently uses a combination of Python (Django) on the backend and React on the frontend. It would be beneficial to continue using this tech stack for Instagram Threads for the sake of consistency and efficiency. 

For the database, Instagram currently uses PostgreSQL, which is known for its reliability and scalability. For hosting and cloud services, Instagram uses Amazon Web Services (AWS), which offers a wide range of services and scalability options.

This tech stack is scalable and reliable, capable of supporting the high levels of traffic that Instagram experiences. It also allows for the seamless integration of Instagram Threads with the existing Instagram app.

## Timeline and Resources

The timeline for the development of Instagram Threads could be as follows:

1. **Phase 1 - Development (3 months):** This phase involves the development of the core features of Instagram Threads. Resources required would include a team of software developers, UX/UI designers, and product managers.

2. **Phase 2 - Testing (1 month):** This phase involves thorough testing of the product to ensure functionality, performance, and security. Resources required would include a team of QA testers and software developers.

3. **Phase 3 - Beta Launch (1 month):** This phase involves launching a beta version of Instagram Threads to a select group of users and collecting feedback. Resources required would include product managers, customer service representatives to handle user feedback, and software developers to make necessary adjustments.

4. **Phase 4 - Refinement (2 months):** This phase involves refining the product based on user feedback and preparing for the full launch. Resources required would include software developers, UX/UI designers, and product managers.

5. **Phase 5 - Full Launch (1 month):** This phase involves launching Instagram Threads to all users and continuing to monitor performance and collect feedback for further improvements. Resources required would include product managers, marketing team to promote the launch, customer service representatives to handle user feedback, and software developers for ongoing maintenance and improvements.

In total, the development of Instagram Threads could take around 8 months from start to finish. The number of employees required for each stage would vary depending on the specific tasks involved, but a rough estimate might be a team of 10-15 people including software developers, UX/UI designers, product managers, QA testers, customer service representatives, and marketing professionals.

